In [1]:
import numpy as np
import os

In [2]:
# load data

# LF input and output
x_l1 = np.loadtxt('./narrow/matter_power_564_Box1000_Part750_15_Box1000_Part3000_z0/train_input_fidelity_0.txt')
y_l1 = np.loadtxt('./narrow/matter_power_564_Box1000_Part750_15_Box1000_Part3000_z0/train_output_fidelity_0.txt')
k_l1 = np.loadtxt('./narrow/matter_power_564_Box1000_Part750_15_Box1000_Part3000_z0/kf.txt')

# x_l2 = np.loadtxt('./narrow/matter_power_564_Box250_Part750_15_Box1000_Part3000_z0/train_input_fidelity_0.txt')
# y_l2 = np.loadtxt('./narrow/matter_power_564_Box250_Part750_15_Box1000_Part3000_z0/train_output_fidelity_0.txt')
# k_l2 = np.loadtxt('./narrow/matter_power_564_Box250_Part750_15_Box1000_Part3000_z0/kf.txt')

# HF input and output
x_h = np.loadtxt('./narrow/matter_power_564_Box250_Part750_15_Box1000_Part3000_z0/train_input_fidelity_1.txt')
y_h = np.loadtxt('./narrow/matter_power_564_Box250_Part750_15_Box1000_Part3000_z0/train_output_fidelity_1.txt')
k_h = np.loadtxt('./narrow/matter_power_564_Box250_Part750_15_Box1000_Part3000_z0/kf.txt')

dir_L1A = "N_L1A_z0"
dir_LHA = 'N_L1HAr_z0'
# dir_LHB = 'N_LHBr_z0'

In [3]:
# L1A

k_cut = np.log10(1)  # cut at k=1, find the closest k value in k_l1 and k_l2


# locate the index of k_cut in k_l1
idx = np.where(k_l1 <= k_cut)[0][-1]

In [4]:
idx

np.int64(40)

In [5]:
# L1 and L2 split

# locate the index of k_mid in k_l1
y_l1A = y_l1[:,:idx]
k_l1A = k_l1[:idx]

In [6]:
# save the split data
# make the directory if not exist

if not os.path.exists(dir_L1A):
    os.makedirs(dir_L1A)


np.savetxt(dir_L1A + '/train_output.txt', y_l1A)

# also copy the input data and k data
np.savetxt(dir_L1A + '/train_input.txt', x_l1)

np.savetxt(dir_L1A + '/kf.txt', k_l1A)


In [7]:
# loop over the L1 input and find the ones (indices) that are in the HF input
indices = []
for i in range(len(x_l1)):
    if x_l1[i] in x_h:
        indices.append(i)


In [8]:
indices

[144, 145, 146, 168, 169, 170, 195, 196, 197, 204, 205, 206, 336, 337, 338]

In [9]:
k_h

array([-1.59976085, -1.56002463, -1.5202884 , -1.48055217, -1.44081595,
       -1.40107972, -1.36134349, -1.32160727, -1.28187104, -1.24213481,
       -1.20239859, -1.16266236, -1.12292613, -1.08318991, -1.04345368,
       -1.00371745, -0.96398123, -0.924245  , -0.88450877, -0.84477255,
       -0.80503632, -0.76530009, -0.72556387, -0.68582764, -0.64609141,
       -0.60635519, -0.56661896, -0.52688273, -0.48714651, -0.44741028,
       -0.40767406, -0.36793783, -0.3282016 , -0.28846538, -0.24872915,
       -0.20899292, -0.1692567 , -0.12952047, -0.08978424, -0.05004802,
       -0.01031179,  0.02942444,  0.06916066,  0.10889689,  0.14863312,
        0.18836934,  0.22810557,  0.2678418 ,  0.30757802,  0.34731425,
        0.38705048,  0.4267867 ,  0.46652293,  0.50625916,  0.54599538,
        0.58573161,  0.62546784,  0.66520406,  0.70494029,  0.74467652,
        0.78441274,  0.82414897,  0.8638852 ,  0.90362142])

In [11]:
# interpolate hf data to l1a k values
from scipy.interpolate import interp1d

f = interp1d(k_h, y_h, axis=1, kind='cubic')

y_hA = f(k_l1A)

In [12]:
y_l1A_H = y_l1A[indices]

In [13]:
# create dataset for LF-HF correlation

# ratio: y_HF / y_LF
# convert back to the original scale
r1A = 10**y_hA / 10**y_l1A_H

In [14]:
# save the dataset

if not os.path.exists(dir_LHA):
    os.makedirs(dir_LHA)

np.savetxt(dir_LHA + '/train_input.txt', x_h)
np.savetxt(dir_LHA + '/train_output.txt', r1A)